##Wczytanie zbioru danych

In [33]:
import pandas as pd 
import numpy as np
import numpy as np
import operator
import math 
import matplotlib.pyplot as plt
data = pd.read_csv('athletes.csv')
data.head()

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze
0,736041664,A Jesus Garcia,ESP,male,10/17/69,1.72,64.0,athletics,0,0,0
1,532037425,A Lam Shin,KOR,female,9/23/86,1.68,56.0,fencing,0,0,0
2,435962603,Aaron Brown,CAN,male,5/27/92,1.98,79.0,athletics,0,0,1
3,521041435,Aaron Cook,MDA,male,1/2/91,1.83,80.0,taekwondo,0,0,0
4,33922579,Aaron Gate,NZL,male,11/26/90,1.81,71.0,cycling,0,0,0


###Przypisanie kolumnom płeć oraz sport odpowiednich wartości klasy:

Kolumna płeć:
*   mężczyźni: klasa 0
*   kobiety: klasa 1

Kolumna sport:


*   gimnastyka: klasa 0
*   koszykówka: klasa 1



In [27]:
data.loc[data.sex == "male", "sex"] = 0
data.loc[data.sex == "female", "sex"] = 1
data.loc[data.sport == "gymnastics", "sport"] = 0
data.loc[data.sport == "basketball", "sport"] = 1

###Losowy podział zbioru danych na dwa podzbiory: testowy i walidacyjny

In [ ]:
data=data[["sex", 'height', 'weight', 'sport']]
indices = np.random.permutation(data.shape[0])
div = int(0.75 * len(indices))
development_id, test_id = indices[:div], indices[div:]
development_set, test_set = data.loc[development_id,:], data.loc[test_id,:]
mean_development_set = development_set.mean()
mean_test_set = test_set.mean()
std_development_set = development_set.std()
std_test_set = test_set.std()
test_class = list(test_set.iloc[:,-1])
dev_class = list(development_set.iloc[:,-1])
mean_development_set = development_set.mean()
mean_test_set = test_set.mean()
std_development_set = development_set.std()
std_test_set = test_set.std()
development_set =development_set.dropna()
test_set =test_set.dropna()


### Wybór wierszy zawierających kolumnę sport z klasy 0 oraz 1

In [30]:
development_set=development_set[(development_set["sport"]==0) 
                          | (development_set["sport"]==1) ]
test_set=test_set[(test_set["sport"]==0) 
                          | (test_set["sport"]==1) ]
index_dev=list(range(0,len(development_set)))
index_test=list(range(0,len(test_set)))
development_set.index=index_dev
test_set.index=index_test

In [31]:
test_set.head(5)

,sex,height,weight,sport
0,1,1.82,70.0,1
1,1,1.56,49.0,0
2,0,1.91,90.0,1
3,1,1.69,55.0,0
4,1,1.73,51.0,0


In [32]:
development_set.head(5)

,sex,height,weight,sport
0,0,2.11,120.0,1
1,0,1.96,90.0,1
2,0,1.92,85.0,1
3,0,1.91,90.0,1
4,1,1.45,46.0,0


### Realizacja klasyfikatora według algorytmu:

1.   Wyliczanie odległości Euklidesowej ze wzoru:
$d(p,q) = d(q,p) = \sqrt{(p_1-p_1)^2 + (q_2-p_2)^2 + ... + (q_n-pn)^2} = \sqrt{\sum^n{i=1}(q_i - p_i)^2}$
2.   Stworzenie data frame z kolumną zawierającą klasy oraz kolumną wyliczonych wartości odległości Euklidesowych
3. Sortowanie data frame ze względu na odległość Euklidesową 
4. Wybór pierwszych k-sąsiadów z kolumny klasy
5. Zliczanie, która wartość (0 lub 1) pojawiła się więcej razy przy danym k
6. Wybór wartości przeważającej i porównanie z wartością odpowiadającą ze zbioru testowego

In [37]:
s=development_set
s=s.drop(columns=['sex', 'height', 'weight', 'sport'])
e_distance = [[] for i in range(len(test_set))] 
result = []
k=3 #liczba sąsiadów
for i in range(len(test_set)):
  for j in range(len(development_set)):
    e_distance[i].append(math.sqrt((test_set.sex[i]-development_set.sex[j])**2
                +(test_set.height[i]-development_set.height[j])**2 
                +(test_set.weight[i]-development_set.weight[j])**2 )) 
  
  s['E_distance']=e_distance[i]  #kolumna z odległością Euklidesową
  s['class_sport']=development_set.sport
  s=s.sort_values('E_distance')
  ind=list(range(0,len(s))) 
  s.index=ind
  ab=s[["class_sport"]][:k]
  idx=ab.class_sport
  q=idx.value_counts() #zliczanie ile razy pojawiły się wartości 0 i 1
  m=q.idxmax() # indeks pod którym znajduje się wartość maksymalna z q
  result.append(m) #dodawanie do listy indeksu (wyniku klasyfikacji)

###Wyniki klasyfikatora

In [49]:
TP=0 #liczba pozytywnie sklasyfikowanych przypadków
FN=0 #liczba błędnie sklasyfikowanych przypadków
TN=0 # liczba przypadków poprawnie niezakfalifikowanych do danej klasy
FP=0 # liczba przypadków pozytywnie zakwalifikowanych do danej klasy podczas gdy do niej nie należą 
for w in range(len(result)):
  if result[w]==test_set.sport[w]:
    TP=TP+1
  elif result[w]==0 and test_set.sport[w]==0:
    TN=TN+1
  elif result[w]==1 and test_set.sport[w]==0:
    FP=FP+1
  else: 
    FN=FN+1
P=round(TP/len(result),2)
R=round(TP/(TP+FN),2)  
A= round((TP+TN)/len(result),2)
F1=round(2*((P*R)/(P+R)),2)
n=[(TP, FN, TN, FP, P, R,A, F1)]
labels=['True Positive', 'False Negative','True Negative', 'False Positive', 'Precision','Accuracy ','Recall', 'F1']
df1=pd.DataFrame.from_records(n, columns=labels)
df1


,True Positive,False Negative,True Negative,False Positive,Precision,Accuracy,Recall,F1
0,151,2,0,3,0.97,0.99,0.97,0.98
